In [1]:
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model.logistic import LogisticRegression
from sklearn import svm

import seaborn as sns


%matplotlib inline

### Load Training Data

In [2]:
train_data = pd.read_csv('data.csv')

### 2 cities - São Gonçalo & Vitória

In [3]:
cities = pd.unique(train_data["city"])
print(cities)

['São Gonçalo' 'Vitória']


## São Gonçalo

In [4]:
train_data_Sao = train_data[train_data["city"] == cities[0]]
train_data_Sao.head()

,Unnamed: 0,wsid,wsnm,elvt,lat,lon,inme,city,prov,mdct,...,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
0,0,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 00:00:00,...,29.7,16.8,25.5,10.8,35,58,32,3.2,101,6.5
1,1,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 01:00:00,...,29.9,13.6,29.0,12.2,39,39,35,3.6,94,6.4
2,2,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 02:00:00,...,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9
3,3,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 03:00:00,...,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8
4,4,178,SÃO GONÇALO,237,-6.835777,-38.311583,A333,São Gonçalo,RJ,2007-11-06 04:00:00,...,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5


#### Data Analyzing

In [5]:
def basic_analyze(df):
    print(df.shape)

    analysis = pd.DataFrame(columns=['col_name', 'nulls', 'uniques', 'dtype'])
    for col in df:
        row = pd.Series({'col_name':col, 
                         'nulls':df[col].isnull().sum(), 
                         'uniques':df[col].unique().size,
                         'dtype':df[col].dtypes})
        analysis = analysis.append(row, ignore_index=True)

    pd.set_option('display.max_rows', 32)
    display(analysis)
    
basic_analyze(train_data_Sao)

(78048, 32)


,col_name,nulls,uniques,dtype
0,Unnamed: 0,0,78048,int64
1,wsid,0,1,int64
2,wsnm,0,1,object
3,elvt,0,1,int64
4,lat,0,1,float64
5,lon,0,1,float64
6,inme,0,1,object
7,city,0,1,object
8,prov,0,1,object
9,mdct,0,78048,object


### Data Cleaning and Preprocessing

#### Drop useless data columns

In [6]:
def dropUselessColumns(df):
    drop_list = []
    for col in df.columns:
        uniques = df[col].unique().size
        if uniques == 1 or uniques == df.shape[0] or col in ['date', 'yr', 'mo', 'da', 'hr']:
            drop_list.append(col)
    
    return df.drop(drop_list, axis=1)
        
pd.set_option('display.max_columns', 30)
train_data_Sao = dropUselessColumns(train_data_Sao)
train_data_Sao.head()

,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
0,NaN,982.5,982.5,981.3,NaN,29.3,12.1,29.7,16.8,25.5,10.8,35,58,32,3.2,101,6.5
1,NaN,983.2,983.2,982.5,NaN,29.0,13.5,29.9,13.6,29.0,12.2,39,39,35,3.6,94,6.4
2,NaN,983.5,983.5,983.2,NaN,27.4,14.0,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9
3,NaN,983.7,983.7,983.4,NaN,25.8,16.9,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8
4,NaN,983.7,983.8,983.6,NaN,25.4,16.4,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5


#### Fill blanks with zero

In [7]:
def fillBlankByZero(df):
    for col in df.columns:
        df[col].fillna(0, inplace=True)

fillBlankByZero(train_data_Sao)
train_data_Sao.head()

,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
0,0.0,982.5,982.5,981.3,0.0,29.3,12.1,29.7,16.8,25.5,10.8,35,58,32,3.2,101,6.5
1,0.0,983.2,983.2,982.5,0.0,29.0,13.5,29.9,13.6,29.0,12.2,39,39,35,3.6,94,6.4
2,0.0,983.5,983.5,983.2,0.0,27.4,14.0,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9
3,0.0,983.7,983.7,983.4,0.0,25.8,16.9,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8
4,0.0,983.7,983.8,983.6,0.0,25.4,16.4,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5


#### Find outliers by z-score  (air pressure related fields, like 'stp', 'smax', 'smin' contain erroneous records)

In [127]:
def getOutlierIndices(df):
    df_zscore = (df - df.mean()) / df.std()
    outliers = set()
    outlier_col = []
    for col in df.columns:
        if col == 'prcp':
            continue   
        ol = list(df_zscore[abs(df_zscore[col]) > 3.5].index)
        if len(ol) > 0:
            outlier_col.append((col, len(ol)))
        outliers.update(ol)
        
    print(outlier_col)
    print(len(outliers))
    return outliers

outliers = getOutlierIndices(train_data_Sao)
train_data_Sao.loc[sorted(outliers)].head()

[('stp', 378), ('smax', 678), ('smin', 593), ('wdsp', 48), ('gust', 53)]
1717


,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
26,0.0,985.4,985.5,985.3,0.000,28.1,14.0,29.3,14.0,28.1,13.1,42,42,37,6.1,120,11.6
74,0.0,982.7,4863.8,982.5,0.000,28.3,15.3,30.6,15.3,28.3,13.6,45,45,35,3.4,102,9.6
83,0.0,2514.0,985.2,984.8,1913.426,27.0,15.2,27.0,16.2,25.3,14.8,49,57,48,3.9,115,7.1
94,0.0,979.8,8384.4,979.0,0.000,25.6,15.5,26.5,18.0,25.1,14.9,54,63,51,1.5,124,2.9
159,0.0,983.7,985.0,8152.0,3710.546,33.4,12.1,33.4,14.2,31.5,11.0,27,35,26,3.6,99,7.9


#### Find rows with all zeros

In [125]:
def getAllZeroIndices(df):
    allZeros = list(df[~((train_data_Sao.T != 0).any())].index)
    print(len(allZeros))
    return allZeros

allZeros = getAllZeroIndices(train_data_Sao)
train_data_Sao.loc[allZeros].head()

10806


,prcp,stp,smax,smin,gbrd,temp,dewp,tmax,dmax,tmin,dmin,hmdy,hmax,hmin,wdsp,wdct,gust
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0
193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0
194,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0.0,0,0.0


#### Prepare X (Past 3 hours data), Y (Precipitation)

In [104]:
def prepareXYByKHours(data, k, removes):
    X_col_names = []
    for i in range(k, 0, -1):
        for col in data.columns:
            if col == 'prcp':
                continue
            X_col_names.append(col + "(<" + str(i) + "hr)")
    
    df_Y = pd.DataFrame(data=data.loc[k:]['prcp'] , columns=['prcp']).reset_index().drop(['index'], axis=1)
    df_X = pd.DataFrame()
    for start in range(k):
        df_X = pd.concat([df_X, data.iloc[start : -k + start, 1:].reset_index()], axis=1).drop(['index'], axis=1)
    df_X.columns = X_col_names
    
    # drop groups with outliers
    removes = set()
    for ol in list(outliers):
        for o in range(max(0, ol - k + 1), min(df_X.shape[0], ol + 1)):
            removes.add(o)
    df_X = df_X.drop(list(removes))
    df_Y = df_Y.drop(list(removes))
        
    return df_X, df_Y

In [131]:
removals = set()
removals.update(outliers, allZeros)
train_X_Sao, train_Y_Sao = prepareXYByKHours(train_data_Sao, 3, removals)

pd.set_option('display.max_columns', 48)
display(pd.concat([train_Y_Sao, train_X_Sao], axis=1))

,prcp,stp(<3hr),smax(<3hr),smin(<3hr),gbrd(<3hr),temp(<3hr),dewp(<3hr),tmax(<3hr),dmax(<3hr),tmin(<3hr),dmin(<3hr),hmdy(<3hr),hmax(<3hr),hmin(<3hr),wdsp(<3hr),wdct(<3hr),gust(<3hr),stp(<2hr),smax(<2hr),smin(<2hr),gbrd(<2hr),temp(<2hr),dewp(<2hr),tmax(<2hr),...,tmin(<2hr),dmin(<2hr),hmdy(<2hr),hmax(<2hr),hmin(<2hr),wdsp(<2hr),wdct(<2hr),gust(<2hr),stp(<1hr),smax(<1hr),smin(<1hr),gbrd(<1hr),temp(<1hr),dewp(<1hr),tmax(<1hr),dmax(<1hr),tmin(<1hr),dmin(<1hr),hmdy(<1hr),hmax(<1hr),hmin(<1hr),wdsp(<1hr),wdct(<1hr),gust(<1hr)
0,0.0,982.5,982.5,981.3,0.000,29.3,12.1,29.7,16.8,25.5,10.8,35,58,32,3.2,101,6.5,983.2,983.2,982.5,0.000,29.0,13.5,29.9,...,29.0,12.2,39,39,35,3.6,94,6.4,983.5,983.5,983.2,0.000,27.4,14.0,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9
1,0.0,983.2,983.2,982.5,0.000,29.0,13.5,29.9,13.6,29.0,12.2,39,39,35,3.6,94,6.4,983.5,983.5,983.2,0.000,27.4,14.0,29.0,...,27.4,13.6,44,44,39,2.5,93,6.9,983.7,983.7,983.4,0.000,25.8,16.9,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8
2,0.0,983.5,983.5,983.2,0.000,27.4,14.0,29.0,14.0,27.4,13.6,44,44,39,2.5,93,6.9,983.7,983.7,983.4,0.000,25.8,16.9,27.4,...,25.8,14.1,58,58,44,1.7,96,5.8,983.7,983.8,983.6,0.000,25.4,16.4,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5
3,0.0,983.7,983.7,983.4,0.000,25.8,16.9,27.4,16.9,25.8,14.1,58,58,44,1.7,96,5.8,983.7,983.8,983.6,0.000,25.4,16.4,26.3,...,25.3,16.4,57,58,56,3.1,110,7.5,983.7,983.8,983.6,0.000,23.8,16.2,25.4,16.4,23.8,16.0,62,62,57,2.0,99,6.8
4,0.0,983.7,983.8,983.6,0.000,25.4,16.4,26.3,17.0,25.3,16.4,57,58,56,3.1,110,7.5,983.7,983.8,983.6,0.000,23.8,16.2,25.4,...,23.8,16.0,62,62,57,2.0,99,6.8,983.7,983.7,983.6,0.000,22.0,16.7,23.8,16.7,22.0,16.2,72,72,62,1.3,93,4.9
5,0.0,983.7,983.8,983.6,0.000,23.8,16.2,25.4,16.4,23.8,16.0,62,62,57,2.0,99,6.8,983.7,983.7,983.6,0.000,22.0,16.7,23.8,...,22.0,16.2,72,72,62,1.3,93,4.9,984.6,984.6,983.7,0.000,19.7,17.4,22.0,17.8,19.5,16.6,86,89,72,0.5,157,2.8
6,0.0,983.7,983.7,983.6,0.000,22.0,16.7,23.8,16.7,22.0,16.2,72,72,62,1.3,93,4.9,984.6,984.6,983.7,0.000,19.7,17.4,22.0,...,19.5,16.6,86,89,72,0.5,157,2.8,985.7,985.7,984.6,0.000,18.3,17.3,19.7,17.3,18.3,16.9,93,94,85,0.0,141,1.5
7,0.0,984.6,984.6,983.7,0.000,19.7,17.4,22.0,17.8,19.5,16.6,86,89,72,0.5,157,2.8,985.7,985.7,984.6,0.000,18.3,17.3,19.7,...,18.3,16.9,93,94,85,0.0,141,1.5,986.7,986.7,985.7,214.149,22.9,18.3,22.9,18.3,18.2,17.1,75,94,75,0.0,248,0.0
8,0.0,985.7,985.7,984.6,0.000,18.3,17.3,19.7,17.3,18.3,16.9,93,94,85,0.0,141,1.5,986.7,986.7,985.7,214.149,22.9,18.3,22.9,...,18.2,17.1,75,94,75,0.0,248,0.0,987.2,987.2,986.7,987.980,25.0,17.1,25.1,18.4,22.9,17.0,61,76,61,3.3,97,6.2
14,0.0,986.0,987.0,986.0,1513.621,31.0,14.3,31.8,16.0,30.0,14.3,36,42,36,3.2,97,9.1,984.8,986.1,984.8,3700.385,32.7,14.0,33.0,...,31.0,13.6,32,37,31,3.8,103,8.6,983.4,984.8,983.4,3484.193,33.7,14.0,34.0,15.6,32.5,12.9,31,34,29,3.7,78,9.6


CPU times: user 384 ms, sys: 124 ms, total: 507 ms
Wall time: 505 ms
